#### Importação das Bibliotecas

In [1]:
# import gspread
# from google.auth.transport.requests import Request
# from google_auth_oauthlib.flow import InstalledAppFlow
# from google.oauth2.credentials import Credentials
# from gspread_dataframe import set_with_dataframe
from datetime import datetime
import pandas as pd
import numpy as np
import requests
import os
from openpyxl import Workbook
from openpyxl.utils.dataframe import dataframe_to_rows
from openpyxl.styles import Font, PatternFill, Alignment, Border, Side

#### Definição das APIs

In [2]:
# URLs das APIs
url_atletas = 'https://api.cartola.globo.com/atletas/pontuados/'
url_partidas = 'https://api.cartola.globo.com/partidas/'
url_mercado = 'https://api.cartola.globo.com/atletas/mercado'
url_clubes = 'https://api.cartola.globo.com/clubes'
url_rodada = 'https://api.cartola.globo.com/rodadas'
url_esquemas = 'https://api.cartola.globo.com/esquemas'
url_posicoes = 'https://api.cartola.globo.com/posicoes'
url_status_mercado = 'https://api.cartola.globo.com/mercado/status'

### Dim Tables

#### Clubes

In [3]:
# Função para obter dados dos clubes
def get_clubes():
    response = requests.get(url_clubes)
    if response.status_code == 200:
        data = response.json()
        if isinstance(data, dict):
            # Criar listas para os dados
            numeros = []
            times = []
            for numero, info in data.items():
                numeros.append(numero)
                times.append(info['nome'])
            # Criar DataFrame
            df_clubes = pd.DataFrame({
                'clube_id': numeros,
                'time': times
            })
            return df_clubes
        else:
            print('Formato de dados inesperado para clubes.')
            return None
    else:
        print(f'Erro na requisição para clubes: {response.status_code}')
        return None

# Obter dados e criar DataFrame
df_clubes = get_clubes()

#### Status dos Jogadores

In [4]:
# Dados de status
status_data = {
    "status": {
        "2": {
            "nome": "Dúvida",
            "id": 2
        },
        "3": {
            "nome": "Suspenso",
            "id": 3
        },
        "5": {
            "nome": "Contundido",
            "id": 5
        },
        "6": {
            "nome": "Nulo",
            "id": 6
        },
        "7": {
            "nome": "Provável",
            "id": 7
        }
    }
}

# Criar DataFrame para status
df_status = pd.DataFrame.from_dict(status_data['status'], orient='index').reset_index()
df_status.rename(columns={'index': 'status_id'}, inplace=True)
df_status = df_status.drop(columns=['id'])

#### Esquemas e Posições

In [5]:
# Funções para obter dados das APIs
def get_esquemas():
    response = requests.get(url_esquemas)
    if response.status_code == 200:
        data = response.json()
        if isinstance(data, list):
            df_esquemas = pd.DataFrame(data)  # Converter a lista de esquemas em DataFrame
            return df_esquemas
        else:
            print('Formato de dados inesperado para esquemas.')
            return None
    else:
        print(f'Erro na requisição para esquemas: {response.status_code}')
        return None

def get_posicoes():
    response = requests.get(url_posicoes)
    if response.status_code == 200:
        data = response.json()
        if isinstance(data, dict):
            df_posicoes = pd.DataFrame(data.values())  # Converter as posições em DataFrame
            return df_posicoes
        else:
            print('Formato de dados inesperado para posições.')
            return None
    else:
        print(f'Erro na requisição para posições: {response.status_code}')
        return None

# Obter os dados das APIs
df_esquemas = get_esquemas()
df_posicoes = get_posicoes()

# Se df_esquemas não for None
if df_esquemas is not None:
    # Obter as abreviações das posições
    posicoes_abreviacoes = df_posicoes['abreviacao'].tolist()
    
    # Inicializar colunas com valores 0
    for posicao in posicoes_abreviacoes:
        df_esquemas[posicao] = 0

    # Preencher as colunas com os valores dos dicionários em 'posicoes'
    for i, row in df_esquemas.iterrows():
        posicoes = row['posicoes']
        for posicao, valor in posicoes.items():
            abreviacao = next((p for p in posicoes_abreviacoes if p in posicao), None)
            if abreviacao:
                df_esquemas.at[i, abreviacao] = valor

    # Remover a coluna original 'posicoes'
    df_esquemas.drop(columns=['posicoes'], inplace=True)

### Fato Tables

#### Histórico Scouts Rodadas

In [6]:
# Mapeamento das letras para descrições de scout
scout_keys = {
    'A': 'assistência',
    'CA': 'cartão amarelo',
    'CV': 'cartão vermelho',
    'DE': 'defesa',
    'DP': 'defesa de pênalti',
    'DS': 'desarme',
    'FC': 'falta cometida',
    'FD': 'finalização defendida',
    'FF': 'finalização pra fora',
    'FS': 'falta sofrida',
    'FT': 'finalização na trave',
    'G': 'gol',
    'GC': 'gol contra',
    'GS': 'gol sofrido',
    'I': 'impedimento',
    'PC': 'pênalti cometido',
    'PP': 'pênalti perdido',
    'PS': 'pênalti sofrido',
    'SG': 'saldo de gol',
    'V': 'vitória'
}

# Função para obter o número de rodadas até a data atual
def get_numero_rodadas():
    response = requests.get(url_rodada)
    if response.status_code == 200:
        data = response.json()
        data_atual = datetime.now()
        rodadas_validas = [rodada['rodada_id'] for rodada in data if datetime.strptime(rodada['fim'], '%Y-%m-%d %H:%M:%S') <= data_atual]
        return rodadas_validas
    else:
        print(f'Erro ao obter o número de rodadas: {response.status_code}')
        return []

# Função para obter dados de atletas pontuados
def get_atletas_pontuados(rodada):
    response = requests.get(f'{url_atletas}{rodada}')
    if response.status_code == 200:
        data = response.json()
        if isinstance(data, dict) and 'atletas' in data:
            df = pd.DataFrame(data['atletas']).T  # Transpor para ter os atletas como linhas
            df.insert(0, 'Rodada', rodada)  # Adicionar a coluna Rodada no início
            return df
        else:
            print(f'Formato de dados inesperado para a rodada {rodada}.')
            return None
    else:
        print(f'Erro na requisição para a rodada {rodada}: {response.status_code}')
        return None

# Obter número de rodadas até a data atual
rodadas = get_numero_rodadas()

# Obter dados para todas as rodadas
dfs = []
for rodada in rodadas:
    df = get_atletas_pontuados(rodada)
    if df is not None:
        # Criar novas colunas para cada tipo de scout
        for key, description in scout_keys.items():
            df[description] = df['scout'].apply(lambda x: x.get(key, 0) if isinstance(x, dict) else 0)
        dfs.append(df)

# Concatenar todos os DataFrames em um só
df_atletas_pontuados_total = pd.concat(dfs, ignore_index=True)

# Ajustar a coluna 'clube_id'
df_atletas_pontuados_total['clube_id'] = df_atletas_pontuados_total['clube_id'].astype(int)
df_clubes['clube_id'] = df_clubes['clube_id'].astype(int)
df_atletas_pontuados_total = df_atletas_pontuados_total.merge(df_clubes[['clube_id', 'time']], on='clube_id', how='left')

# Reorganizar o DataFrame com as colunas na ordem especificada
df_atletas_pontuados_total = df_atletas_pontuados_total[
    ['Rodada', 'apelido', 'time', 'pontuacao', 'posicao_id',
       'clube_id', 'entrou_em_campo', 'assistência', 'cartão amarelo',
       'cartão vermelho', 'defesa', 'defesa de pênalti', 'desarme',
       'falta cometida', 'finalização defendida', 'finalização pra fora',
       'falta sofrida', 'finalização na trave', 'gol', 'gol contra',
       'gol sofrido', 'impedimento', 'pênalti cometido', 'pênalti perdido',
       'pênalti sofrido', 'saldo de gol', 'vitória']
]

#### Mercado Atual

In [7]:
# Lista de IDs de clubes únicos
clube_ids = df_atletas_pontuados_total['clube_id'].unique()

# Lista para armazenar DataFrames individuais
dfs = []

# Função para obter dados do mercado
def get_mercado_data(clube_id):
    response = requests.get(f'{url_mercado}/{clube_id}')
    if response.status_code == 200:
        data = response.json()
        return data
    else:
        print(f'Erro na requisição para clube_id {clube_id}: {response.status_code}')
        return None

# Obter dados para cada clube_id e organizar os dados em DataFrame
for clube_id in clube_ids:
    mercado_data = get_mercado_data(clube_id)
    if mercado_data:
        atletas = mercado_data.get('atletas', [])
        if isinstance(atletas, list):
            # Criar DataFrame para os atletas
            df_atletas = pd.DataFrame(atletas)
            # Adicionar a coluna 'clube_id'
            df_atletas['clube_id'] = clube_id
            # Adicionar DataFrame à lista
            dfs.append(df_atletas)

# Concatenar todos os DataFrames em um só
df_mercado = pd.concat(dfs, ignore_index=True)

# Reorganizar o DataFrame conforme necessário
df_mercado = df_mercado[['atleta_id', 'apelido', 'nome', 'foto', 'clube_id', 'posicao_id', 'status_id', 
                         'pontos_num', 'preco_num', 'variacao_num', 'media_num', 'jogos_num', 
                         'entrou_em_campo']]

#### Histórico de Partidas

In [8]:
# Função para obter o número de rodadas até a data atual
def get_numero_rodadas():
    response = requests.get(url_rodada)
    if response.status_code == 200:
        data = response.json()
        data_atual = datetime.now()
        rodadas_validas = [rodada['rodada_id'] for rodada in data if datetime.strptime(rodada['fim'], '%Y-%m-%d %H:%M:%S') <= data_atual]
        return rodadas_validas
    else:
        print(f'Erro ao obter o número de rodadas: {response.status_code}')
        return []

# Função para obter dados de partidas de uma rodada específica
def get_partidas(rodada):
    response = requests.get(f'{url_partidas}{rodada}')
    if response.status_code == 200:
        data = response.json()
        if isinstance(data, dict) and 'partidas' in data:
            df = pd.DataFrame(data['partidas'])  # Converter a lista de partidas em DataFrame
            df.insert(0, 'Rodada', rodada)  # Inserir a coluna 'Rodada' no início
            return df
        else:
            print(f'Formato de dados inesperado para partidas na rodada {rodada}.')
            return None
    else:
        print(f'Erro na requisição para partidas na rodada {rodada}: {response.status_code}')
        return None

# Obter o número de rodadas até a data atual
rodadas = get_numero_rodadas()

# Obter dados de partidas para todas as rodadas
dfs_partidas = []
for rodada in rodadas:
    df_partida = get_partidas(rodada)
    if df_partida is not None:
        dfs_partidas.append(df_partida)

# Concatenar todos os DataFrames de partidas em um só
df_partidas_total = pd.concat(dfs_partidas, ignore_index=True)

df_partidas_total = df_partidas_total.merge(df_clubes[['time', 'clube_id']], left_on='clube_casa_id', right_on='clube_id', how='left')
df_partidas_total = df_partidas_total.merge(df_clubes[['time', 'clube_id']], left_on='clube_visitante_id', right_on='clube_id', how='left')

# Renomear colunas para melhor identificação
df_partidas_total['time_casa'] = df_partidas_total['time_x']
df_partidas_total['time_visitante'] = df_partidas_total['time_y']

#### Próxima Rodada

In [9]:
# Requisição para obter o histórico de rodadas
response_rodadas = requests.get(url_rodada)
rodadas_data = response_rodadas.json()

# Criar um DataFrame com o histórico de rodadas
df_rodadas = pd.DataFrame(rodadas_data)

# Loop para iterar sobre todas as rodadas e coletar dados de partidas
all_partidas = []

for rodada in df_rodadas['rodada_id']:
    # URL para obter dados de partidas da rodada atual
    url_partidas = f'https://api.cartola.globo.com/partidas/{rodada}'
    
    # Requisição para obter os dados da rodada
    response_partidas = requests.get(url_partidas)
    partidas_data = response_partidas.json()

    # Extraindo a lista de partidas
    partidas = partidas_data.get('partidas', [])
    
    # Criar DataFrame com os dados das partidas e adicionar a coluna de rodada
    df_partidas = pd.DataFrame(partidas)
    df_partidas['rodada'] = rodada
    
    # Adicionar ao conjunto de todas as partidas
    all_partidas.append(df_partidas)

# Concatenar todas as rodadas em um único DataFrame
df_rodadas = pd.concat(all_partidas, ignore_index=True)

# Selecionar apenas as colunas de interesse
df_rodadas = df_rodadas[['rodada', 'clube_casa_id', 'clube_visitante_id']]

# Merge para obter os nomes dos clubes
df_rodadas = df_rodadas.merge(df_clubes[['time', 'clube_id']], left_on='clube_casa_id', right_on='clube_id', how='left')
df_rodadas = df_rodadas.merge(df_clubes[['time', 'clube_id']], left_on='clube_visitante_id', right_on='clube_id', how='left')

# Renomear colunas para melhor identificação
df_rodadas['time_casa'] = df_rodadas['time_x']
df_rodadas['time_visitante'] = df_rodadas['time_y']

# Selecionar as colunas finais
df_rodadas = df_rodadas[['rodada', 'clube_casa_id', 'clube_visitante_id', 'time_casa', 'time_visitante']]

C:\Users\u612567\AppData\Local\Temp\ipykernel_29968\2711517480.py:30: FutureWarning: The behavior of DataFrame concatenation with empty or all-NA entries is deprecated. In a future version, this will no longer exclude empty or all-NA columns when determining the result dtypes. To retain the old behavior, exclude the relevant entries before the concat operation.
  df_rodadas = pd.concat(all_partidas, ignore_index=True)


#### Salvar Raw Data

In [10]:
# Função para adicionar uma planilha ao workbook com personalização
def add_sheet_to_workbook(wb, df, sheet_name):
    ws = wb.create_sheet(title=sheet_name)
    
    # Adicionar o cabeçalho
    for r_idx, row in enumerate(dataframe_to_rows(df, index=False, header=True)):
        for c_idx, value in enumerate(row):
            cell = ws.cell(row=r_idx+1, column=c_idx+1)
            cell.value = str(value) if value is not None else ''  # Garantir que o valor seja uma string
            
            if r_idx == 0:  # Cabeçalho
                cell.font = Font(name='Arial', size=11, bold=True, color="FFFFFF")
                cell.fill = PatternFill(start_color="FFA500", end_color="FFA500", fill_type="solid")
            else:  # Dados
                cell.font = Font(name='Arial', size=11)
                cell.alignment = cell.alignment.copy(horizontal='center')

    # Ajustar largura das colunas
    for col in ws.columns:
        max_length = 0
        column = col[0].column_letter  # Obtém a letra da coluna
        for cell in col:
            try:
                cell_value = str(cell.value)
                if len(cell_value) > max_length:
                    max_length = len(cell_value)
            except:
                pass
        adjusted_width = (max_length + 2)
        ws.column_dimensions[column].width = adjusted_width

# Criar um workbook
wb = Workbook()
wb.remove(wb.active)  # Remove a planilha padrão criada por default

# Adicionar cada DataFrame como uma planilha
dfs = {
    "Partidas Totais": df_partidas_total,
    "Mercado": df_mercado,
    "Atletas Pontuados Total": df_atletas_pontuados_total,
    "Rodadas": df_rodadas,
    "Esquemas": df_esquemas,
    "Posições": df_posicoes,
    "Status": df_status,
    "Clubes": df_clubes
}

for sheet_name, df in dfs.items():
    add_sheet_to_workbook(wb, df, sheet_name)

# Salvar o arquivo Excel
wb.save("Cartola_RawData.xlsx")

C:\Users\u612567\AppData\Local\Temp\ipykernel_29968\387976282.py:16: DeprecationWarning: Call to deprecated function copy (Use copy(obj) or cell.obj = cell.obj + other).
  cell.alignment = cell.alignment.copy(horizontal='center')


## Análise

#### Tabela Brasileirão-Cartola

In [11]:
# Ajustar a função para incluir os nomes dos times como uma coluna
def calcular_estatisticas(df):
    # Inicializar tabela para todos os times
    tabela = pd.DataFrame()

    # Calcular vitórias, empates e derrotas para mandantes e visitantes
    tabela['Jogos'] = df.groupby('time_casa').size() + df.groupby('time_visitante').size()
    tabela['Gols Pro'] = df.groupby('time_casa')['placar_oficial_mandante'].sum() + df.groupby('time_visitante')['placar_oficial_visitante'].sum()
    tabela['Gols Contra'] = df.groupby('time_casa')['placar_oficial_visitante'].sum() + df.groupby('time_visitante')['placar_oficial_mandante'].sum()
    tabela['Saldo de Gols'] = tabela['Gols Pro'] - tabela['Gols Contra']

    # Calcular vitórias
    tabela['Vitorias'] = df[df['placar_oficial_mandante'] > df['placar_oficial_visitante']].groupby('time_casa').size() + df[df['placar_oficial_visitante'] > df['placar_oficial_mandante']].groupby('time_visitante').size()

    # Calcular empates
    tabela['Empates'] = df[df['placar_oficial_mandante'] == df['placar_oficial_visitante']].groupby('time_casa').size() + df[df['placar_oficial_mandante'] == df['placar_oficial_visitante']].groupby('time_visitante').size()

    # Calcular derrotas
    tabela['Derrotas'] = tabela['Jogos'] - tabela['Vitorias'] - tabela['Empates']

    # Calcular pontos
    tabela['Pontos'] = tabela['Vitorias'] * 3 + tabela['Empates']

    # Calcular aproveitamento
    tabela['Aproveitamento (%)'] = (tabela['Pontos'] / (tabela['Jogos'] * 3)) * 100

    # Organizar colunas no formato desejado
    tabela = tabela[['Pontos', 'Jogos', 'Vitorias', 'Empates', 'Derrotas', 'Gols Pro', 'Gols Contra', 'Saldo de Gols', 'Aproveitamento (%)']]

    # Preencher com 0 os NaN que podem aparecer em times que ainda não jogaram
    tabela.fillna(0, inplace=True)

    # Resetar o índice para que os nomes dos times virem uma coluna
    tabela.reset_index(inplace=True)

    # Renomear a coluna do índice para 'Time'
    tabela.rename(columns={'index': 'Time'}, inplace=True)

    return tabela

# Gerar a tabela do campeonato
tabela_brasileirao = calcular_estatisticas(df_partidas_total)

# Ordenar pela pontuação e exibir a tabela final
tabela_brasileirao = tabela_brasileirao.sort_values(by='Pontos', ascending=False)

#### Médias e Totais de Scouts por Time

In [12]:
# Estatísticas relevantes
stats = ['assistência', 'cartão amarelo', 'cartão vermelho', 'defesa', 'defesa de pênalti', 
         'desarme', 'falta cometida', 'finalização defendida', 'finalização pra fora', 
         'falta sofrida', 'finalização na trave', 'gol', 'gol contra', 'gol sofrido', 
         'impedimento', 'pênalti cometido', 'pênalti perdido', 'pênalti sofrido', 
         'saldo de gol', 'vitória']

# Calcular a soma das estatísticas por posicao_id, time e rodada
df_soma_time_pos = df_atletas_pontuados_total.groupby(['posicao_id', 'time', 'clube_id', 'Rodada'])[stats].sum().reset_index()

# Calcular o total de rodadas jogadas por time
df_rodadas_time = df_atletas_pontuados_total.groupby('time')['Rodada'].nunique().reset_index()
df_rodadas_time.rename(columns={'Rodada': 'total_rodadas'}, inplace=True)

# Calcular a soma total dos eventos por posicao_id e time
df_total_time_pos = df_atletas_pontuados_total.groupby(['posicao_id', 'time'])[stats].sum().reset_index()

# Adicionar o número de rodadas jogadas ao DataFrame total
df_total_time_pos = df_total_time_pos.merge(df_rodadas_time, on='time', how='left')

# Calcular a média dos eventos por jogo para cada posição e time
df_media_time_pos = df_total_time_pos.copy()
for stat in stats:
    df_media_time_pos[stat + '_media_por_jogo'] = df_media_time_pos[stat] / df_media_time_pos['total_rodadas']

# Manter apenas as colunas de média
df_media_time_pos = df_media_time_pos[['posicao_id', 'time'] + [stat + '_media_por_jogo' for stat in stats]]

# Calcular a soma total dos eventos por time
df_total_time = df_atletas_pontuados_total.groupby('time')[stats].sum().reset_index()
df_total_time = df_total_time.merge(df_rodadas_time, on='time', how='left')

# Calcular a média dos eventos por jogo para cada time
df_media_time = df_total_time.copy()
for stat in stats:
    df_media_time[stat + '_media_por_jogo'] = df_media_time[stat] / df_media_time['total_rodadas']

# Manter apenas as colunas de média
df_media_time = df_media_time[['time'] + [stat + '_media_por_jogo' for stat in stats]]

df_media_time = df_media_time.merge(df_clubes[['time', 'clube_id']], left_on='time', right_on='time', how='left')
df_total_time = df_total_time.merge(df_clubes[['time', 'clube_id']], left_on='time', right_on='time', how='left')

#### Totais de Scouts por Time e Rodada

In [13]:
# Passo 1: Calcular a soma das estatísticas por time e por rodada
df_soma_time_rodada = df_atletas_pontuados_total.groupby(['time', 'Rodada'])[stats].sum().reset_index()

# Passo 2: Adicionar o número de rodadas jogadas ao DataFrame total por time
# Usando o df_rodadas_time para calcular o número de rodadas jogadas por time
df_rodadas_time_rodada = df_atletas_pontuados_total.groupby(['time', 'Rodada'])['Rodada'].count().reset_index(name='total_rodadas')

# Merge para adicionar o número de rodadas ao df_soma_time_rodada
df_soma_time_rodada = df_soma_time_rodada.merge(df_rodadas_time_rodada, on=['time', 'Rodada'], how='left')
df_soma_time_rodada = df_soma_time_rodada.merge(df_clubes[['time', 'clube_id']], left_on='time', right_on='time', how='left')

#### Totais de Scouts por Time nas X últimas Rodadas

In [14]:
# Passo 1: Identificar as últimas 5 rodadas para cada time
def ultimas_rodadas_por_time(df):
    # Ordenar por time e rodada, e pegar as últimas 5 rodadas para cada time
    return df.groupby('clube_id')['Rodada'].apply(lambda x: x.sort_values(ascending=False).unique()[:5])

# Aplicar a função para obter as últimas 5 rodadas para cada time
ultimas_rodadas = ultimas_rodadas_por_time(df_soma_time_rodada)

# Passo 2: Filtrar o DataFrame para essas últimas 5 rodadas
df_soma_ultimas_rodadas = df_soma_time_rodada[df_soma_time_rodada.apply(
    lambda row: row['Rodada'] in ultimas_rodadas[row['clube_id']], axis=1)]

# Passo 3: Calcular as somas dos scouts para as últimas 5 rodadas
df_estatisticas_ultimas = df_soma_ultimas_rodadas.groupby('clube_id').sum().reset_index()

df_estatisticas_ultimas = df_estatisticas_ultimas.merge(df_clubes[['time', 'clube_id']], left_on='clube_id', right_on='clube_id', how='left')
df_estatisticas_ultimas['time'] = df_estatisticas_ultimas['time_y'] 

# Exibir o DataFrame com as estatísticas das últimas 5 rodadas
df_estatisticas_ultimas = df_estatisticas_ultimas[['clube_id', 'time', 'assistência', 'cartão amarelo',
       'cartão vermelho', 'defesa', 'defesa de pênalti', 'desarme',
       'falta cometida', 'finalização defendida', 'finalização pra fora',
       'falta sofrida', 'finalização na trave', 'gol', 'gol contra',
       'gol sofrido', 'impedimento', 'pênalti cometido', 'pênalti perdido',
       'pênalti sofrido', 'saldo de gol', 'vitória']]

#### Médias e Totais de Scouts por Jogador

In [15]:
# Estatísticas relevantes
stats = ['assistência', 'cartão amarelo', 'cartão vermelho', 'defesa', 'defesa de pênalti', 
         'desarme', 'falta cometida', 'finalização defendida', 'finalização pra fora', 
         'falta sofrida', 'finalização na trave', 'gol', 'gol contra', 'gol sofrido', 
         'impedimento', 'pênalti cometido', 'pênalti perdido', 'pênalti sofrido', 
         'saldo de gol', 'vitória']

# Calcular a soma das estatísticas por posicao_id, time e rodada
df_soma_atletas_rodada = df_atletas_pontuados_total.groupby(['apelido', 'time', 'posicao_id','clube_id', 'Rodada'])[stats].sum().reset_index()
df_soma_atletas_scouts = df_atletas_pontuados_total.groupby(['apelido', 'time', 'posicao_id','clube_id'])[stats].sum().reset_index()

# Definir o número de rodadas
num_rodadas = df_atletas_pontuados_total['Rodada'].max()

# Filtrar os dados para as últimas 5 rodadas
rodadas_filtro = range(num_rodadas - 4, num_rodadas + 1)  # Considera as últimas 5 rodadas
df_ultimas_5_rodadas = df_atletas_pontuados_total[df_atletas_pontuados_total['Rodada'].isin(rodadas_filtro)]

# Calcular a soma dos scouts apenas nas últimas 5 rodadas
df_soma_ultimas_5_rodadas = df_ultimas_5_rodadas.groupby(['apelido', 'time', 'posicao_id', 'clube_id', 'Rodada'])[stats].sum().reset_index()

# Calcular a média ao longo do campeonato para cada jogador (considerando todas as rodadas)
df_media_scouts = df_atletas_pontuados_total.groupby(['apelido', 'time', 'posicao_id', 'clube_id'])[stats].mean().reset_index()
df_media_scouts_5_rodadas = df_ultimas_5_rodadas.groupby(['apelido', 'time', 'posicao_id', 'clube_id'])[stats].mean().reset_index()

#### Comparações Próxima Rodada

In [16]:
df_comparacoes = df_rodadas.copy()

# Merge com estatísticas dos times da casa (usando clube_casa_id e rodada)
df_comparacoes = df_comparacoes.merge(
    df_soma_time_rodada, 
    left_on=['clube_casa_id', 'rodada'], 
    right_on=['clube_id', 'Rodada'], 
    how='left', 
    suffixes=('', '_casa')
)

# Merge com estatísticas dos times visitantes (usando clube_visitante_id e rodada)
df_comparacoes = df_comparacoes.merge(
    df_soma_time_rodada, 
    left_on=['clube_visitante_id', 'rodada'], 
    right_on=['clube_id', 'Rodada'], 
    how='left', 
    suffixes=('', '_visitante')
)

# Atualizar a coluna 'resultado' com base em novas comparações
df_comparacoes['resultado'] = np.where(df_comparacoes['gol'].isna() | df_comparacoes['gol_visitante'].isna(), 
                                       np.nan, 
                                       np.where(df_comparacoes['gol'] > df_comparacoes['gol_visitante'], 
                                                'Vitoria_casa',
                                       np.where(df_comparacoes['gol'] < df_comparacoes['gol_visitante'], 
                                                'Vitoria_visitante',
                                                'Empate')))

df_comparacoes['resultado'] = df_comparacoes['resultado'].replace('nan', np.nan)
df_historico = df_comparacoes[df_comparacoes['resultado'].notna()]

#### Salvar Excel de Análises

In [17]:
# Função para adicionar uma planilha ao workbook com personalização especial para a tabela do Brasileirão
def add_sheet_to_workbook_brasileirao(wb, df, sheet_name):
    ws = wb.create_sheet(title=sheet_name)
    
    # Definir os limites para classificação
    limite_libertadores = 6   # Times até a 6ª posição vão à Libertadores
    limite_sulamericana = 12  # Times da 7ª à 12ª posição vão à Sul-Americana
    limite_rebaixados = 16     # Times da 17ª à 20ª posição são rebaixados
    
    # Cores suaves
    verde_suave = "B2E7B2"  # Verde suave para Libertadores
    amarelo_suave = "FFF2CC"  # Amarelo suave para Sul-Americana
    vermelho_suave = "F4CCCC"  # Vermelho suave para rebaixados

    # Borda suave
    thin_border = Border(left=Side(style='thin'),
                         right=Side(style='thin'),
                         top=Side(style='thin'),
                         bottom=Side(style='thin'))

    # Remover linhas de grade da planilha
    ws.sheet_view.showGridLines = False

    # Adicionar o cabeçalho
    for r_idx, row in enumerate(dataframe_to_rows(df, index=False, header=True)):
        for c_idx, value in enumerate(row):
            cell = ws.cell(row=r_idx+1, column=c_idx+1)
            cell.value = str(int(value)) if isinstance(value, (int, float)) else str(value)  # Remove casas decimais
            
            # Adicionar borda suave
            cell.border = thin_border

            if r_idx == 0:  # Cabeçalho
                cell.font = Font(name='Arial', size=10, bold=True, color="FFFFFF")
                cell.fill = PatternFill(start_color="002052", end_color="002052", fill_type="solid")
                cell.alignment = Alignment(horizontal='center', vertical='center')
            else:  # Dados
                cell.font = Font(name='Arial', size=10)
                cell.alignment = Alignment(horizontal='center', vertical='center')

                # Aplicar destaques para as posições com cores suaves
                if r_idx <= limite_libertadores:  # Libertadores (1º a 6º)
                    cell.fill = PatternFill(start_color=verde_suave, end_color=verde_suave, fill_type="solid")
                elif limite_libertadores < r_idx <= limite_sulamericana:  # Sul-Americana (7º a 12º)
                    cell.fill = PatternFill(start_color=amarelo_suave, end_color=amarelo_suave, fill_type="solid")
                elif r_idx > limite_rebaixados:  # Rebaixados (17º a 20º)
                    cell.fill = PatternFill(start_color=vermelho_suave, end_color=vermelho_suave, fill_type="solid")

    # Ajustar largura das colunas
    for col in ws.columns:
        max_length = 0
        column = col[0].column_letter  # Obtém a letra da coluna
        for cell in col:
            try:
                cell_value = str(cell.value)
                if len(cell_value) > max_length:
                    max_length = len(cell_value)
            except:
                pass
        adjusted_width = (max_length + 2)
        ws.column_dimensions[column].width = adjusted_width

    # Ajustar a altura das linhas para melhorar o espaçamento visual
    for row in ws.iter_rows():
        ws.row_dimensions[row[0].row].height = 18  # Definindo altura das linhas

# Criar um workbook
wb = Workbook()
wb.remove(wb.active)  # Remove a planilha padrão criada por default

# Adicionar cada DataFrame como uma planilha
dfs = {
    "Tabela": tabela_brasileirao,
    "Média Scouts Times": df_media_time,
    "Total Scouts Times": df_total_time,
    "Total Scouts por Time e Rodada": df_soma_time_rodada,
    "Totais de Scouts por Time nas 5 últimas Rodadas": df_estatisticas_ultimas,
    "Total de Scouts por Jogador e Rodada": df_soma_atletas_rodada,
    "Total de Scouts por Jogador": df_soma_atletas_scouts
}

# Adicionar o dataframe personalizado com destaques para a tabela do Brasileirão
add_sheet_to_workbook_brasileirao(wb, tabela_brasileirao, "Tabela Brasileirão")

# Adicionar as outras planilhas normalmente
for sheet_name, df in dfs.items():
    if sheet_name != "Tabela":
        add_sheet_to_workbook(wb, df, sheet_name)

# Salvar o arquivo Excel
wb.save("Cartola_Analise.xlsx")

C:\Users\u612567\AppData\Local\Temp\ipykernel_29968\387976282.py:16: DeprecationWarning: Call to deprecated function copy (Use copy(obj) or cell.obj = cell.obj + other).
  cell.alignment = cell.alignment.copy(horizontal='center')
C:\Users\u612567\AppData\Local\anaconda3\Lib\site-packages\openpyxl\workbook\child.py:99: UserWarning: Title is more than 31 characters. Some applications may not be able to read the file
  warnings.warn("Title is more than 31 characters. Some applications may not be able to read the file")
